# Backtesting ML Classification-Based

In [ ]:
???

![](src/res_classification.png)

## Load the model

In [ ]:
import pickle

In [ ]:
with open('???.pkl', '???') as f:
    model_dt = pickle.load(f)

In [ ]:
model_dt

## Load the data

In [ ]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

## Backtesting.py Library

### Create your Strategy Class

In [ ]:
from backtesting import Backtest, Strategy

In [ ]:
Backtest(???)

#### Simulate the prediction for the last observation

#### Write the prediction process in the Strategy class

In [ ]:
class ???(Strategy):
    def init(self):
        self.model = ???

    def next(self):
        explanatory_today = ???
        forecast_tomorrow = ???
        
        # conditions to sell or buy

#### Calculate Purchase Recommendation

##### Buy if it goes up

##### Buy if it goes and sell if down

> You can only sell if you have already bought

#### Add conditions to the strategy

### Define initial conditions

In [ ]:
bt = Backtest(???, ???,
              cash=???, commission=.002, exclusive_orders=True)

### Run backtesting

### Interpret backtesting results

In [ ]:
results.to_frame(name='Values').loc[:'Return [%]']

## Practice to master the knowledge

Work on the challenge with another dataset:

1. Learn the <a>mental models</a> to solve the challenge faster.
2. Complete the <a href="03C_Backtesting ML Classification-Based.ipynb">notebook</a>.